In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your original `dataset.csv`


Saving dataset.csv to dataset (1).csv


In [ ]:
import pandas as pd
df = pd.read_csv("dataset.csv")

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)

traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5, 'cycle': 0.3}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_map)

df[["Timestamp", "SystemCodeNumber", "Occupancy", "Capacity", "QueueLength", "TrafficLevel", "IsSpecialDay", "VehicleTypeWeight"]].to_csv("parking_stream_model2.csv", index=False)


In [ ]:
!pip install pathway --quiet


In [ ]:
import pathway as pw

class ParkingSchema2(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficLevel: int
    IsSpecialDay: int
    VehicleTypeWeight: float


In [ ]:
data2 = pw.demo.replay_csv("parking_stream_model2.csv", schema=ParkingSchema2, input_rate=1000)

data2 = data2.with_columns(
    t = data2.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)

BASE_PRICE = 10.0
LAMBDA = 0.5
ALPHA = 3.0
BETA = 0.5
GAMMA = 2.0
DELTA = 2.5
EPSILON = 1.5
MIN_PRICE = 5.0
MAX_PRICE = 20.0

def compute_demand(occ, cap, queue, traffic, special, veh_weight):
    raw = (
        ALPHA * (occ / cap)
        + BETA * queue
        - GAMMA * traffic
        + DELTA * special
        + EPSILON * veh_weight
    )
    return max(0, min(raw / 20, 1))

model2_with_demand = data2.with_columns(
    demand = pw.apply(compute_demand,
                      data2.Occupancy,
                      data2.Capacity,
                      data2.QueueLength,
                      data2.TrafficLevel,
                      data2.IsSpecialDay,
                      data2.VehicleTypeWeight)
)

model2_output = model2_with_demand.with_columns(
    price = pw.apply(lambda d: max(MIN_PRICE, min(MAX_PRICE, BASE_PRICE * (1 + LAMBDA * d))),
                     model2_with_demand.demand)
)



In [ ]:
pw.io.csv.write(
    model2_output.select(
        timestamp = model2_output.Timestamp,
        lot_id = model2_output.SystemCodeNumber,
        occupancy = model2_output.Occupancy,
        capacity = model2_output.Capacity,
        queue = model2_output.QueueLength,
        traffic = model2_output.TrafficLevel,
        special = model2_output.IsSpecialDay,
        vehicle = model2_output.VehicleTypeWeight,
        demand = model2_output.demand,
        price = model2_output.price
    ),
    "model2_prices.csv"
)

pw.run()


Output()

In [ ]:
import pandas as pd

df_model2 = pd.read_csv("model2_prices.csv")
df_model2['timestamp'] = pd.to_datetime(df_model2['timestamp'])
df_model2 = df_model2.sort_values('timestamp')
df_model2.head()


,timestamp,lot_id,occupancy,capacity,queue,traffic,special,vehicle,demand,price,time,diff
0,2016-10-04 07:59:00,BHMNCPHST01,237,1200,2,1,0,0.5,0.017125,10.085625,1751745222916,1
1,2016-10-04 07:59:00,BHMNCPNST01,249,485,2,1,0,1.0,0.102010,10.510052,1751745222916,1
2,2016-10-04 07:59:00,BHMEURBRD01,117,470,2,1,0,1.0,0.062340,10.311702,1751745222916,1
3,2016-10-04 07:59:00,BHMMBMMBX01,264,687,2,1,0,1.0,0.082642,10.413210,1751745222916,1
4,2016-10-04 07:59:00,BHMBCCMKT01,61,577,1,1,0,1.0,0.015858,10.079289,1751745222916,1


In [ ]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
from bokeh.palettes import Category10

output_notebook()

# Load Model 2 output
df = pd.read_csv("model2_prices.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')


In [ ]:
plots = []
lots = df['lot_id'].unique()

for i, lot in enumerate(lots[:10]):  # Show up to 10 lots
    lot_df = df[df['lot_id'] == lot]
    source = ColumnDataSource(lot_df)

    p = figure(
        x_axis_type="datetime",
        title=f"Model 2 Pricing – Lot {lot}",
        x_axis_label="Time",
        y_axis_label="Price ($)",
        width=800,
        height=300
    )

    color = Category10[10][i % 10]

    p.line(x='timestamp', y='price', source=source, color=color, line_width=2, legend_label="Model 2 Price")
    p.circle(x='timestamp', y='price', source=source, size=5, color=color)

    p.add_tools(HoverTool(tooltips=[
        ("Time", "@timestamp{%F %T}"),
        ("Price", "@price{$0.00}"),
        ("Demand", "@demand{0.00}"),
        ("Occupancy", "@occupancy"),
        ("Queue", "@queue"),
        ("Traffic", "@traffic"),
        ("Special Day", "@special"),
        ("Vehicle Weight", "@vehicle")
    ], formatters={"@timestamp": "datetime"}))

    p.legend.location = "top_left"
    p.legend.click_policy = "hide"
    p.toolbar.logo = None
    p.toolbar.autohide = True
    plots.append(p)

show(column(*plots))


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

lot_df = df[df['lot_id'] == lot_id]
lot_df = lot_df.sort_values('timestamp')

source = ColumnDataSource(lot_df)

p = figure(x_axis_type='datetime', title=f"Model 2 Pricing – Lot {lot_id}",
           x_axis_label='Time', y_axis_label='Price ($)', width=800, height=300)

p.line(x='timestamp', y='price', source=source, line_width=2, color='blue')
p.scatter(x='timestamp', y='price', source=source, size=6, color='blue', marker="circle")

show(p)
